**Transformer实战（上）**

# Transformer案例实战 之 机器翻译

机器翻译是一种使用计算机程序将一种语言的文字自动翻译成另一种语言的技术，在Transformer的应用中拥有重要的地位。这个任务涉及到从源语言中抽取意义，并将这些意义准确地表达在目标语言中，同时尽可能保持原文的风格和语境。现代的机器翻译系统通常采用Transformer为核心的深度学习技术，通过训练大量的双语数据集来学习语言之间的映射关系。这些系统能够理解和翻译复杂的句子结构，适应不同的语言特点，并处理词义多变的情况。机器翻译是自然语言处理领域中的一个重要而活跃的研究方向，广泛应用于全球化交流、多语种内容创作和国际贸易等领域。

对于Transformer模型来说，机器翻译是一个典型的序列到序列（Seq2Seq）的有监督任务。这意味着模型需要接收一系列的输入（源语言的文本），并输出另一系列的文本（目标语言的翻译）。Transformer中的自注意力机制允许模型在翻译时能够关注到输入序列中的所有单词，帮助模型理解更复杂的语言结构和语境依赖，使得翻译更加准确和流畅；同时，在机器翻译任务重，**Transformer中的编码器用于理解输入文本，而解码器则用于生成翻译文本**。每一层都进一步提炼和传递信息，增强了模型在处理复杂文字时的能力。

在今天的项目中，我们将完成一个英文到中文的翻译任务。通常来说，一个机器翻译任务需要覆盖至少如下的流程：

- 一、数据准备

对于任何NLP任务来说，我们都需要对数据进行详细的处理、毕竟算法本身无法处理文字数据，而语言数据同时带有时序属性和文字属性，具有复杂的处理流程。

1. **环境设置**：定义了工作目录和模型目录，用于存储数据和模型检查点。

2. **数据加载**：准备英文和中文的句子文件，并确认这两个文件的行数相同，保证每行英文句子对应一行中文翻译。机器翻译模型通常需要规模较大、同时也足够精确的翻译数据来进行训练，这部分数据通常需要人为进行翻译来构建。当然现在在有人类审核的情况下，我们可以通过大语言模型来辅助我们生成一个个的翻译数据集；在经典的深度学习领域，也有一些常用的机器翻译数据集——

> 中英数据集:
> - LDC (Linguistic Data Consortium) 数据集：LDC提供了多种中英双语数据集，这些数据集广泛用于学术研究和商业应用。
> - CWMT (China Workshop on Machine Translation)：CWMT是中国的机器翻译评测活动，提供了一些中英平行语料库，包括新闻、法律文件等多个领域。

> 中法数据集:
> - United Nations Parallel Corpus：虽然这个数据集涵盖多种语言，但其中也包含大量的中法对照材料，适合用于法律和国际关系领域的翻译训练。

> 中日数据集:
> - Tanaka Corpus：这是一个开源的中日双语语料库，主要包含日常用语，适合用于基础对话和日常交流的翻译训练。
> - JEC Basic Sentence Data：这个数据集包含基础的中日句子对，适用于初学者和基础翻译模型的训练。

> 中韩数据集:
> - KAIST Korean-Chinese Parallel Corpus：由韩国科学技术院（KAIST）提供，包括韩语和中文的平行文本，适用于科技和教育领域的翻译。
> - Naver Labs’ Multilingual Corpus：包含中韩以及其他语言对，适合用于多语言翻译系统的开发和测试。

当然，也还有很多涉及其他语言之间互相翻译的经典数据集——

> Europarl：包含欧洲议会的会议记录，涵盖多种欧洲语言。这个数据集非常适合训练政治和法律文本的翻译模型。

> WMT (Workshop on Machine Translation)：每年都会发布新的数据集，用于机器翻译的国际比赛。这些数据集包含多种语言对，涉及新闻和其他类型的文本，非常适合用来评估和比较不同的翻译算法。

> UN Parallel Corpus：联合国文件的平行语料库，包括六种官方语言：英语、法语、西班牙语、俄语、阿拉伯语和中文。这个数据集特别适合用来训练和测试法律和外交文本的翻译模型。

> IWSLT (International Workshop on Spoken Language Translation)：主要关注口语翻译，包含TED演讲等的多语种数据，适合用来训练口语风格的翻译模型。

> Tatoeba：一个包含简短句子的多语种平行语料库，适用于基础语言学习和测试。

3. **分词**：在获得数据集之后，首先我们要进行分词。对于英文、法文这些天然就有空格来进行分词的语言来说，是否进行分词或许没有那么重要，但对于中文、日语这些没有天然分分隔的语言来说，分词是无法跳过的步骤。分词，又叫做Tokenizer，意思是将文字分成最小语义单元Token。你或许注意到过，基于Transformer的GPT、BERT等结构是“一个词一个词往外蹦”着进行预测的，其实就是一个token、一个token地进行预测的。因此，分词即是为了输入，也是为了输出。

在深度学习和最近的NLP模型中，如BERT和GPT系列模型，采用了一种称为subword tokenization的方法，例如Byte-Pair Encoding (BPE) 或 SentencePiece。这些方法在某种程度上可以绕过传统分词的需求，通过将词汇分解成更小的、意义相关的片段（subwords），既保留了词汇的部分语义，也能有效处理未知或罕见词汇的问题。在本次的案例中，我们需要将采用`transformers`库的`AutoTokenizer`和`BertTokenizer`对英文进行分词，中文分词则通过字符级分割（一个字是一个Token）。

4. **构建词典**：当我们将句子进行分词之后，下一步最好可以构建当前句子所构成的词典。词典不是一个必备流程，但是一个优化的流程，我们可以利用词典创造和embedding不一样的数据编码结果，丰富数据编码的形式，也可以直接在构建好的词典的基础上进行embedding。**在使用pytorch实现序列到序列（Seq2Seq）任务的时候，nn.Transformer中是不包括位置编码、掩码等encoder、decoder之前的结构的；因此我们首先要将数据传入embedding和位置编码结构，然后才会将数据传入Transformer，我们在构建词典后、真正获得的数据是与词典紧密相连的单词索引，而非直接放入Transoformer的数据**。我们通常会基于相同的词典将训练集、测试集进行索引，同时编码器和解码器都会基于相同的词典将单词索引映射为embedding后的向量，以确保输入和输出的一致性。在本次案例中，我们将利用`torchtext.vocab`的`build_vocab_from_iterator`函数，基于分词结果构建英文和中文的词典。

> - **经过词典所构建的数据（也就是单词索引）一般是什么样的？**

源语言的单词索引序列通常是二维的数据结构。每个维度代表的含义如下：

第一维代表批次中的句子数量（也就是batch size）。这允许模型同时处理多个句子，提高处理效率。

第二维代表每个句子中的单词索引。句子长度可以是固定的（通过截断或填充来标准化），这样每个批次的数据形状才能保持一致。

每个单词索引是一个整数，表示在词典中的位置。这里有一个简单的Markdown代码例子来表示源语言的单词索引序列。假设我们有以下中文句子，已经过分词和转换为索引：

句子1: "我 爱 北京 天安门"
句子2: "北京 是 中国 的 首都"

假设词典中每个词对应的索引如下：

我 -> 1, 爱 -> 2, 北京 -> 3, 天安门 -> 4, 是 -> 5, 中国 -> 6, 的 -> 7, 首都 -> 8

如果将这些句子转换为单词索引序列，并考虑到填充（以保持统一长度，这里填充的索引为0），则可得到：

In [ ]:
句子1索引: [1, 2, 3, 4]

句子2索引: [3, 5, 6, 7, 8]

为了在一个批次中处理，假设填充每个句子到最大长度5，则单词索引序列为：

In [ ]:
[[1, 2, 3, 4, 0],  # 句子1填充后

 [3, 5, 6, 7, 8]]  # 句子2

结构为（2,5），代表一共有2个句子，句子的最大长度为5。

- 二、模型构建

在之前的课程中，我们详细复现过Transformer架构。在构建Transformer模型的过程中，首先要确定模型的极大核心结构，包括位置编码层、编码器层、解码器层、自注意力机制层、前馈网络等等；然后要确定模型的关键结构参数，如编解码器层数、每层的维度和头的数量，尤其是必须理清每个神经网络结构的输入和输出数据状态。

1. **位置编码**：
在Transformer模型中，位置编码用于向模型提供关于单词在句子中位置的信息，因为模型的自注意力层本身不包含处理序列顺序的机制。位置编码通过将位置信息编码为向量并与词向量相加，使得模型能够考虑到词语的顺序关系，从而理解语言的语法和句子结构。这样的设计允许Transformer有效地处理自然语言，并保持在不同位置的词语能够被区分和正确解释。我们定义了`PositionalEncoding`类为Transformer模型提供位置信息，这对于处理序列数据至关重要。

2. **模型定义**：在我们的案例中，我们使用了一个基本的Transformer模型，命名为`TranslationModel`类，它包括源语言嵌入、目标语言嵌入、位置编码、Transformer层和最终的预测层。对于机器翻译这样seq2seq的任务来说，编码器的输入数据结构为源语言的单词索引序列，解码器的数据数据结构为目标语言的单词索引序列，在seq2seq过程中，一般Transformer对输入数据的处理过程是：
> **文本转索引**：原始文本通过查找构建好的词典被转换为单词索引序列。这一步是将自然语言中的单词转换为模型可以理解的数值形式。<br><br>
> **索引转嵌入**：这些单词索引接着被用来从嵌入层中获取对应的词向量。这一步是通过词嵌入将每个单词的索引映射到一个高维空间中，以便捕捉和表达词汇的语义和语法属性。<br><br>
> **加入位置编码**：在获取词向量后，模型还会加入位置编码，以引入序列中各单词的位置信息，这对于帮助模型理解词语之间的关系非常重要。

Transformer模型在执行机器翻译任务时，编码器负责进行源语言的信息提取，解码器负责输出翻译的内容，这个过程是有先后顺序之分的。回顾一下Transformer的原理，解码器在输出内容时，会需要编码器输出的原文的内容、同时也会需要掩码后正确标签的内容（如果你无法理解这个，回去看Transformer原理）、或者自己已经生成的内容（例如，在生成第n个词时，解码器会考虑到第1到n-1个已生成的词。当然，大部分时候，解码器会考虑的是正确的标签中的1到n-1个词）。因此解码器必须在编码器结束运行之后才能开始运行，这使得编码器会先输入数据、运行完毕后，解码器才会开始输入数据。

在实际的代码实现中，这种先后顺序通常体现在数据流的处理上。首先，源语言的数据被送入编码器，编码器处理后的输出被存储起来。接着，这些编码器的输出连同解码器的初始输入（通常是一个起始符号）一起送入解码器。解码器基于这些输入逐步生成目标语言的输出。在某些实现中，这个过程可能在一个大的循环中反复执行，特别是在使用教师强制（teacher forcing）的训练过程中，解码器的每一步输出都可能作为下一步的输入。

需要注意的是，由于Transformer模型是一个字、一个字地进行输入，因此也是一个字一个字地进行输出。为了判断当前句子中下一个最合适的字是什么，**无论是编码器还是解码器、输出的结构都是一个向量，通常表示为对词汇表的一个概率分布，即每个词/字作为句子中正确的下一个词/字的概率**。最终，这些概率分布被用来生成目标语言的文本，通常是通过选取每一步概率最高的单词。

3. **损失函数**：在我们的案例中，我们使用`KLDivLoss`进行损失计算，适用于处理预测分布与实际分布的差异。KLDivLoss，即Kullback-Leibler散度损失，是一种在机器学习中常用于衡量两个概率分布差异的损失函数。它主要用于比较目标概率分布 Q 和预测概率分布 P 之间的相对熵，也就是这两个分布的差异程度。

$$D_{KL}(P \parallel Q) = \sum_i Q(i) \log \left(\frac{Q(i)}{P(i)}\right)$$

其中：
- P是模型预测的概率分布，通常是对数概率形式。
- Q是目标或真实概率分布。
- i索引覆盖所有可能的事件。
- Q(i)和P(i)分别是在分布 Q 和 P 中第 i 个事件的概率。

如果 P 是模型输出的对数概率形式，公式可以调整为：

$$D_{KL}(P \parallel Q) = \sum_i Q(i) (\log Q(i) - P(i))$$

在机器翻译任务中使用`KLDivLoss`作为损失函数可以非常有效，主要原因在于机器翻译中的输出通常被表示为词汇表上的概率分布。每一个输出词汇的概率反映了在给定的上下文中该词被选择作为翻译的可能性。`KLDivLoss`可以衡量模型生成的概率分布与目标概率分布之间的距离，它允许模型学习到更细致的概率差异，尤其是当目标分布在某些词汇上具有高度确定性时，KLDivLoss 可以强调这些词汇的重要性，从而指导模型更准确地预测目标语言中每个词的概率。

除了 KLDivLoss，机器翻译过程中还可以使用其他几种损失函数：
> - 交叉熵损失（Cross Entropy Loss）：这是机器翻译中最常用的损失函数之一。它测量的是预测概率分布与实际概率分布之间的差异，特别适用于分类问题，其中输出是一个离散的类别变量（在机器翻译中即词汇表中的单词）。<br><br>
> - 困惑度（Perplexity）：虽然困惑度本身不是一个损失函数，但它是评价语言模型性能的一种标准，常常用来衡量模型在机器翻译任务中的表现。低困惑度意味着模型对数据的预测更加准确。<br><br>
> - 标签平滑（Label Smoothing）：这是一种正则化技术，通常与交叉熵损失结合使用。通过给非目标标签分配一个小的非零概率，它可以帮助改善模型的泛化能力，减少模型对某些频繁标签的过度自信。<br><br>
> - L2损失：虽然不常见，但在一些特定情况下，如模型输出与目标输出之间的误差是连续的，也可以考虑使用L2损失。

- 三、训练过程

相比起数据和架构定义的过程，机器翻译的训练过程对我们来说相对简单和熟悉。我们需要定义Dataloader等结构，并且需要保持每个batch内的数据长度一致（需要对短的数据进行填充，对长的数据进行裁剪）。除此之外，我们需要定义优化器、进行epoch上的循环等过程。这个流程与其他深度学习算法并无区别，因此对大家来说相对容易。

1. **数据加载器**：在PyTorch定义`DataLoader`用于批量加载数据，以及`collate_fn`函数对数据进行适当处理，确保批次内的数据长度一致。
2. **优化器**：我们选择Adam优化器进行参数优化。
3. **训练循环**：进行模型训练，包括前向传播、损失计算、梯度计算和参数更新。同时设置定期保存模型的逻辑。

- 四、推理和翻译
1. **翻译函数**：对于Transformer来说，输出的结构是一个个的概率分布，因此我们还需要将该概率分布转化为具体的文字。因此我们需要定义`translate`函数，将训练好的模型打包、接受一个英文句子作为输入，逐词生成其中文翻译。
2. **逐步预测**：从`<bos>`开始，逐步通过模型预测下一个词，直到生成`<eos>`或达到最大句子长度。

接下来就让我们一步步来完成这个案例。

## 数据准备

### 环境设置

In [1]:
#导入所需的库
import os
import math

import torch
import torch.nn as nn
# hugging face的分词器，github地址：https://github.com/huggingface/tokenizers
from tokenizers import Tokenizer
# 用于构建词典
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn.functional import pad, log_softmax
from pathlib import Path
from tqdm import tqdm

在这里，对你来说可能需要全新安装的库有torchtext，tensorboard以及tqdm。你可以使用下面的代码来进行安装。需要注意的是，torchtext需要与你的pytorch版本进行匹配后才能安装成功，匹配表如下：

In [4]:
import torch

print(torch.__version__)

2.2.0+cu121


![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/Case/01.png)

In [10]:
!pip install torchtext==0.17.0


   ---------------------------------------- 1.9/1.9 MB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 1.3/1.3 MB 650.2 kB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.2.0
    Uninstalling torchtext-0.2.0:
      Successfully uninstalled torchtext-0.2.0


In [3]:
!pip install tensorboard

   ---------------------------------------- 5.5/5.5 MB 982.4 kB/s eta 0:00:00
   ---------------------------------------- 3.8/3.8 MB 644.8 kB/s eta 0:00:00
   -------------------------------------- 105.4/105.4 kB 674.8 kB/s eta 0:00:00


如我们的流程所示，我们首先进行各种环境设置——

In [16]:
# 工作目录、为Transformer放数据、缓存文件的目录
work_dir = Path("./dataset")
# 训练好的模型会放在该目录下，注意隔一段时间就要对模型进行保存，这是深度学习训练的基本
model_dir = Path(r"D:\pythonwork\2024DL\model")
# 上次运行到的地方，如果是第一次运行，为None，如果中途暂停了，下次运行时，指定目前最新的模型即可。
model_checkpoint = None # 'model_10000.pt'

# 如果工作目录不存在，则创建一个
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

# 如果模型目录不存在，则创建一个
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# 英文句子的文件路径/数据集路径，
en_filepath = './dataset/train.en'
# 中文句子的文件路径/数据集路径
zh_filepath = './dataset/train.zh'

- 查看数据与数据的属性 之 查看数据本身

In [3]:
# 查看一下文件中的数据
# 打开并读取英文文件
with open(en_filepath, 'r', encoding='utf-8') as file:
    # 读取所有行
    english_sentences = file.readlines()
    # 打印前5行，假设我们只想看前几行
    print("English sentences (first 5 lines):")
    for line in english_sentences[:5]:
        print(line.strip())

English sentences (first 5 lines):
A pair of red - crowned cranes have staked out their nesting territory
A pair of crows had come to nest on our roof as if they had come for Lhamo.
A couple of boys driving around in daddy's car.
A pair of nines? You pushed in with a pair of nines?
Fighting two against one is never ideal,


In [4]:
# 打开并读取中文文件
with open(zh_filepath, 'r', encoding='utf-8') as file:
    # 读取所有行
    chinese_sentences = file.readlines()
    # 打印前5行，同样只看前几行
    print("\nChinese sentences (first 5 lines):")
    for line in chinese_sentences[:5]:
        print(line.strip())


Chinese sentences (first 5 lines):
一对丹顶鹤正监视着它们的筑巢领地
一对乌鸦飞到我们屋顶上的巢里，它们好像专门为拉木而来的。
一对乖乖仔开着老爸的车子。
一对九？一对九你就全下注了？
一对二总不是好事，


- 查看数据与数据的属性 之 确保两种语言的样本量一致

In [5]:
# 定义一个获取文件行数的方法
# 在开始训练之前，一定要确保数据集中两种语言的句子是一一对应的
def get_row_count(filepath):
    count = 0
    for _ in open(filepath, encoding='utf-8'):
        count += 1
    return count

# 英文句子数量
en_row_count = get_row_count(en_filepath)
# 中文句子数量
zh_row_count = get_row_count(zh_filepath)

In [6]:
en_row_count #一共有1000w个句子

10000000

In [7]:
# 验证，如果句子数量不一致，则主动报错
assert en_row_count == zh_row_count, "英文和中文文件行数不一致！"
# 句子数量，主要用于后面显示进度。
row_count = en_row_count

In [8]:
en_row_count == zh_row_count #Assert是一种让代码主动发起报错的经典Python用法

True

- 查看数据与数据的属性 之 确定每个句子的长度，方便进行裁剪和填充

In [9]:
import random

def sample_file_statistics(filepath, sample_fraction=0.01):
    # 初始化统计变量
    total_length = 0
    length_square_sum = 0
    max_length = 0
    count = 0
    
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            if random.random() < sample_fraction:
                # 计算行长度
                length = len(line.strip())
                # 累加总长度
                total_length += length
                # 累加长度的平方，用于方差计算
                length_square_sum += length ** 2
                # 更新最大长度
                max_length = max(max_length, length)
                # 计数
                count += 1

    # 计算平均长度和方差
    if count > 0:
        average_length = total_length / count
        variance = (length_square_sum - (total_length ** 2) / count) / count
    else:
        average_length = 0
        variance = 0

    return average_length, max_length, variance

# 抽样并计算英文和中文句子的平均长度、最大长度和方差
en_stats = sample_file_statistics(en_filepath)
zh_stats = sample_file_statistics(zh_filepath)

print(f"英文句子 平均长度 = {en_stats[0]:.2f}, 最大长度 = {en_stats[1]}, 方差 = {en_stats[2]:.2f}")
print(f"中文句子: 平均长度 = {zh_stats[0]:.2f}, 最大长度 = {zh_stats[1]}, 方差 = {zh_stats[2]:.2f}")

英文句子 平均长度 = 54.03, 最大长度 = 571, 方差 = 1693.09
中文句子: 平均长度 = 17.21, 最大长度 = 205, 方差 = 165.29


In [24]:
# 定义句子最大长度，如果句子不够这个长度，则填充，若超出该长度，则裁剪
# 我们的数据集中方差超级大，所以一般会设置超出平均长度一些
# 如果你需要更强大的模型，你应该顺应最大长度
max_length = 72
print("句子数量为：", en_row_count)
print("句子最大长度为：", max_length)

# 定义英文和中文词典，都为Vocab类对象，后面会对其初始化
en_vocab = None
zh_vocab = None

# 定义batch_size，由于是训练文本，占用内存较小，可以适当大一些
batch_size = 64
# epochs数量，不用太大，因为句子数量较多，甚至可以设置为1
# 越大的模型、收到epochs的影响越小，在许多大语言模型的训练环境中，我们都是使用epoch=1的情况
epochs = 1
# 多少步保存一次模型，防止程序崩溃导致模型丢失。
save_after_step = 5000

# 是否使用缓存，由于文件较大，初始化动作较慢，所以将初始化好的文件持久化
use_cache = True

# 定义训练设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("batch_size:", batch_size)
print("每{}步保存一次模型".format(save_after_step))
print("Device:", device)

句子数量为： 10000000
句子最大长度为： 72
batch_size: 64
每5000步保存一次模型
Device: cuda


### 分词与词典构建

我们要从huggingface库中直接调出相应的分词器来用，分词器是自然语言处理中用来将原始文本转换为模型可以理解的格式的工具。它通常执行以下任务：

- 分词（Tokenization）：将连续的文本字符串分割成离散的单元（tokens），例如单词、子词或符号。
  
- 添加特殊标记：例如，为句子添加开始 [CLS] 和结束 [SEP] 标记，这在某些模型如BERT中是必需的。
  
- 生成注意力掩码：告诉模型哪些部分是真实数据，哪些部分是填充。
  
- 转换为ID：将每个token转换为词汇表中的唯一ID。

在这里我们使用的 AutoTokenizer 是 huggingface的transformers 库中的一个工具，可以自动根据给定的预训练模型名称加载相应的分词器。这样做的好处是你不需要知道背后具体的分词器细节，只需要提供模型的名称。

注意，huggingface的使用需要全局代理魔法。没有魔法或魔法不合格，则会报`MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443),Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))`错误。如果尝试调用太多次，也会报上述错误。

In [14]:
!ping huggingface.co #使用这段代码来验证你是否能连得上huggingface，如果你的代码显示下面的内容，证明你无法连接huggingface


正在 Ping huggingface.co [157.240.7.8] 具有 32 字节的数据:
请求超时。
请求超时。
请求超时。
请求超时。

157.240.7.8 的 Ping 统计信息:
    数据包: 已发送 = 4，已接收 = 0，丢失 = 4 (100% 丢失)，


如果你无法连接huggingface，我的建议是你直接访问huggingface.co的网页端，并在网页端将你所需的模型放到本地。你可以参考这篇文章，或者直接使用镜像网站hf-mirror:

> https://zhuanlan.zhihu.com/p/689290456

> https://hf-mirror.com/

In [44]:
# 加载基础的分词器模型，使用的是基础的bert模型。`uncased`意思是不区分大小写
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = Tokenizer.from_pretrained("bert-base-uncased")

def en_tokenizer(line):
    """
    定义英文分词器，后续也要使用
    :param line: 一句英文句子，例如"I'm learning Deep learning."
    :return: subword分词后的记过，例如：['i', "'", 'm', 'learning', 'deep', 'learning', '.']
    """
    # 使用bert进行分词，并获取tokens。add_special_tokens是指不要在结果中增加‘<bos>’和`<eos>`等特殊字符
    return tokenizer.encode(line, add_special_tokens=False).tokens

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000017F49F4FA30>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 23ebcfa7-57c0-4a0f-a925-dc5909b1f69b)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tokenizer.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000180FDB8DC60>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 40961f3b-4d34-4831-a93e-83ed13278223)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json


即便导入成功，在多次访问后依然会出现maxretry的错误 ↑ 因此最好是将模型下载到本地后，直接从本地进行调用。

In [45]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def en_tokenizer(line):
    return tokenizer.convert_ids_to_tokens(tokenizer.encode(line,  add_special_tokens=False))

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000017F49F4F1C0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 3b939734-2337-4b97-b73c-e8b230da758a)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt


看一下英语的tokenizer分词的结果：

In [46]:
print(en_tokenizer("I'm a English tokenizer."))

['i', "'", 'm', 'a', 'english', 'token', '##izer', '.']


接下来，我们需要根据分好的词语来建立词典。在这里需要注意的是，在定义分词函数时，我们需要使用yield关键字来帮助我们节约内存。yield是一个非常有用的关键字，它可以允许你在创建函数时将函数设置成类似迭代器的形式，这样的函数在每次被调用时不需要一次加载整个数据集到内存中，可以不断地访问大型数据集中的片段数据。这对于处理大数据文件特别有用，因为它可以显著减少内存使用。

In [47]:
def yield_en_tokens():
    """
    每次yield一个分词后的英文句子，之所以yield方式是为了节省内存。
    如果先分好词再构造词典，那么将会有大量文本驻留内存，造成内存溢出。
    """
    file = open(en_filepath, encoding='utf-8')
    print("-------开始构建英文词典-----------")
    for line in tqdm(file, desc="构建英文词典", total=row_count):
        yield en_tokenizer(line)
    file.close()

接下来我们结合缓存机制、构造英文词典：

In [48]:
# 指定英文词典缓存文件路径
en_vocab_file = work_dir / "vocab_en.pt"
# 如果使用缓存，且缓存文件存在，则加载缓存文件
if use_cache and os.path.exists(en_vocab_file):
    en_vocab = torch.load(en_vocab_file, map_location="cpu")
# 否则就从0开始构造词典
else:
    # 构造词典
    en_vocab = build_vocab_from_iterator(
        # 传入一个可迭代的token列表。例如[['i', 'am', ...], ['machine', 'learning', ...], ...]
        yield_en_tokens(),
        # 最小频率为2，即一个单词最少出现两次才会被收录到词典
        min_freq=2,
        # 在词典的最开始加上这些特殊token
        specials=["<s>", "</s>", "<pad>", "<unk>"],
    )
    # 设置词典的默认index，后面文本转index时，如果找不到，就会用该index填充
    en_vocab.set_default_index(en_vocab["<unk>"])
    # 保存缓存文件
    if use_cache:
        torch.save(en_vocab, en_vocab_file)

In [49]:
# 打印一下看一下效果，用前10个词为例
print("英文词典大小:", len(en_vocab))
print(dict((i, en_vocab.lookup_token(i)) for i in range(10)))

英文词典大小: 27584
{0: '<s>', 1: '</s>', 2: '<pad>', 3: '<unk>', 4: '.', 5: ',', 6: 'the', 7: "'", 8: 'i', 9: 'you'}


接下来是中文词典的构造：

In [50]:
def zh_tokenizer(line):
    """
    定义中文分词器
    :param line: 中文句子，例如：机器学习
    :return: 分词结果，例如['机','器','学','习']
    """
    return list(line.strip().replace(" ", ""))


def yield_zh_tokens():
    file = open(zh_filepath, encoding='utf-8')
    for line in tqdm(file, desc="构建中文词典", total=row_count):
        yield zh_tokenizer(line)
    file.close()

In [51]:
zh_vocab_file = work_dir / "vocab_zh.pt"
if use_cache and os.path.exists(zh_vocab_file):
    zh_vocab = torch.load(zh_vocab_file, map_location="cpu")
else:
    zh_vocab = build_vocab_from_iterator(
        yield_zh_tokens(),
        min_freq=1,
        specials=["<s>", "</s>", "<pad>", "<unk>"],
    )
    zh_vocab.set_default_index(zh_vocab["<unk>"])
    torch.save(zh_vocab, zh_vocab_file)

In [52]:
# 打印看一下效果，用前10个词为例
print("中文词典大小:", len(zh_vocab))
print(dict((i, zh_vocab.lookup_token(i)) for i in range(10)))

中文词典大小: 8280
{0: '<s>', 1: '</s>', 2: '<pad>', 3: '<unk>', 4: '。', 5: '的', 6: '，', 7: '我', 8: '你', 9: '是'}


如此我们的词典就构造完成了。接下来我们将上述所有流程打包在一个继承自Dataset类的pytorch数据读取类中，帮助我们将数据以tensor的形式呈现给pytorch算法：

### 数据批量加载为tensor

In [53]:
class TranslationDataset(Dataset):

    def __init__(self):
        # 加载英文tokens
        self.en_tokens = self.load_tokens(en_filepath, en_tokenizer, en_vocab, "构建英文tokens", 'en')
        # 加载中文tokens
        self.zh_tokens = self.load_tokens(zh_filepath, zh_tokenizer, zh_vocab, "构建中文tokens", 'zh')

    def __getitem__(self, index):
        return self.en_tokens[index], self.zh_tokens[index]

    def __len__(self):
        return row_count

    def load_tokens(self, file, tokenizer, vocab, desc, lang):
        """
        加载tokens，即将文本句子们转换成index们。
        :param file: 文件路径，例如"./dataset/train.en"
        :param tokenizer: 分词器，例如en_tokenizer函数
        :param vocab: 词典, Vocab类对象。例如 en_vocab
        :param desc: 用于进度显示的描述，例如：构建英文tokens
        :param lang: 语言。用于构造缓存文件时进行区分。例如：’en‘
        :return: 返回构造好的tokens。例如：[[6, 8, 93, 12, ..], [62, 891, ...], ...]
        """

        # 定义缓存文件存储路径
        cache_file = work_dir / "tokens_list.{}.pt".format(lang)
        # 如果使用缓存，且缓存文件存在，则直接加载
        if use_cache and os.path.exists(cache_file):
            print(f"正在加载缓存文件{cache_file}, 请稍后...")
            return torch.load(cache_file, map_location="cpu")

        # 从0开始构建，定义tokens_list用于存储结果
        tokens_list = []
        # 打开文件
        with open(file, encoding='utf-8') as file:
            # 逐行读取
            for line in tqdm(file, desc=desc, total=row_count):
                # 进行分词
                tokens = tokenizer(line)
                # 将文本分词结果通过词典转成index
                tokens = vocab(tokens)
                # append到结果中
                tokens_list.append(tokens)
        # 保存缓存文件
        if use_cache:
            torch.save(tokens_list, cache_file)

        return tokens_list

In [54]:
dataset = TranslationDataset()

正在加载缓存文件dataset\tokens_list.en.pt, 请稍后...
正在加载缓存文件dataset\tokens_list.zh.pt, 请稍后...


查看编码后的句子：

In [55]:
print(dataset.__getitem__(0))

([11, 2730, 12, 554, 19, 17210, 18077, 27, 3078, 203, 57, 102, 18832, 3653], [12, 40, 1173, 1084, 3169, 164, 693, 397, 84, 100, 14, 5, 1218, 2397, 535, 67])


现在，我们要定义一个函数，将短句子进行填充、长句子进行裁剪：

In [56]:
def collate_fn(batch):
    """
    将dataset的数据进一步处理，并组成一个batch。
    :param batch: 一个batch的数据，例如：
                  [([6, 8, 93, 12, ..], [62, 891, ...]),
                  ....
                  ...]
    :return: 填充后的且等长的数据，包括src, tgt, tgt_y, n_tokens
             其中src为原句子，即要被翻译的句子
             tgt为目标句子：翻译后的句子，但不包含最后一个token
             tgt_y为label：翻译后的句子，但不包含第一个token，即<bos>
             n_tokens：tgt_y中的token数，<pad>不计算在内。
    """

    # 定义'<bos>'的index，在词典中为0，所以这里也是0
    bs_id = torch.tensor([0])
    # 定义'<eos>'的index
    eos_id = torch.tensor([1])
    # 定义<pad>的index
    pad_id = 2

    # 用于存储处理后的src和tgt
    src_list, tgt_list = [], []

    # 循环遍历句子对儿
    for (_src, _tgt) in batch:
        """
        _src: 英语句子，例如：`I love you`对应的index
        _tgt: 中文句子，例如：`我 爱 你`对应的index
        """

        processed_src = torch.cat(
            # 将<bos>，句子index和<eos>拼到一块
            [
                bs_id,
                torch.tensor(
                    _src,
                    dtype=torch.int64,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    _tgt,
                    dtype=torch.int64,
                ),
                eos_id,
            ],
            0,
        )

        """
        将长度不足的句子进行填充到max_padding的长度的，然后增添到list中

        pad：假设processed_src为[0, 1136, 2468, 1349, 1]
             第二个参数为: (0, 72-5)
             第三个参数为：2
        则pad的意思表示，给processed_src左边填充0个2，右边填充67个2。
        最终结果为：[0, 1136, 2468, 1349, 1, 2, 2, 2, ..., 2]
        """
        src_list.append(
            pad(
                processed_src,
                (0, max_length - len(processed_src),),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_length - len(processed_tgt),),
                value=pad_id,
            )
        )

    # 将多个src句子堆叠到一起
    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)

    # tgt_y是目标句子去掉第一个token，即去掉<bos>
    tgt_y = tgt[:, 1:]
    # tgt是目标句子去掉最后一个token
    tgt = tgt[:, :-1]

    # 计算本次batch要预测的token数
    n_tokens = (tgt_y != 2).sum()

    # 返回batch后的结果
    return src, tgt, tgt_y, n_tokens

在Dataloader中将我们的数据集进行打包：

In [57]:
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [58]:
src, tgt, tgt_y, n_tokens = next(iter(train_loader))
src, tgt, tgt_y = src.to(device), tgt.to(device), tgt_y.to(device)

最终全部裁剪到72个token以下：

In [59]:
print("src.size:", src.size())
print("tgt.size:", tgt.size())
print("tgt_y.size:", tgt_y.size())
print("n_tokens:", n_tokens)

src.size: torch.Size([64, 72])
tgt.size: torch.Size([64, 71])
tgt_y.size: torch.Size([64, 71])
n_tokens: tensor(1266)


正如我们所说，此时transformer的数据输入结构为二维的单词索引，每个batch有64个句子，每个句子最长包含72个单词的索引。

## 模型构建

### 位置编码

首先我们来进行位置编码、注意力机制、编码器、解码器等结构的构建。这些内容都与我们在之前课程中、复现transformer时的结构相一致，因此在这里我们就不进行详细的解读了：

In [21]:
class PositionalEncoding(nn.Module):
    "进行位置编码."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 初始化Shape为(max_len, d_model)的PE (positional encoding)
        pe = torch.zeros(max_len, d_model).to(device)
        # 初始化一个tensor [[0, 1, 2, 3, ...]]
        position = torch.arange(0, max_len).unsqueeze(1)
        # 这里就是sin和cos括号中的内容，通过e和ln进行了变换
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        # 计算PE(pos, 2i)
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算PE(pos, 2i+1)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 为了方便计算，在最外面在unsqueeze出一个batch
        pe = pe.unsqueeze(0)
        # 如果一个参数不参与梯度下降，但又希望保存model的时候将其保存下来
        # 这个时候就可以用register_buffer
        self.register_buffer("pe", pe)

    def forward(self, x):
        """
        x 为embedding后的inputs，例如(1,7, 128)，batch size为1,7个单词，单词维度为128
        """
        # 将x和positional encoding相加。
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

### Embedding与掩码

In [23]:
class TranslationModel(nn.Module):

    def __init__(self, d_model, src_vocab, tgt_vocab, dropout=0.1):
        super(TranslationModel, self).__init__()

        # 定义原句子的embedding
        # embedding的维度被设置成了超参数
        # 在本次案例中我们使用的是256
        self.src_embedding = nn.Embedding(len(src_vocab), d_model, padding_idx=2)
        # 定义目标句子的embedding
        self.tgt_embedding = nn.Embedding(len(tgt_vocab), d_model, padding_idx=2)
        # 定义posintional encoding
        self.positional_encoding = PositionalEncoding(d_model, dropout, max_len=max_length)
        # 定义Transformer
        self.transformer = nn.Transformer(d_model, dropout=dropout, batch_first=True)

        # 定义最后的预测层，这里并没有定义Softmax，而是把他放在了模型外。
        self.predictor = nn.Linear(d_model, len(tgt_vocab))

    def forward(self, src, tgt):
        """
        进行前向传递，输出为Decoder的输出。注意，这里并没有使用self.predictor进行预测，
        因为训练和推理行为不太一样，所以放在了模型外面。
        :param src: 原batch后的句子，例如[[0, 12, 34, .., 1, 2, 2, ...], ...]
        :param tgt: 目标batch后的句子，例如[[0, 74, 56, .., 1, 2, 2, ...], ...]
        :return: Transformer的输出，或者说是TransformerDecoder的输出。
        """

        """
        生成tgt_mask，即阶梯型的mask，例如：
        [[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]]
        tgt.size()[-1]为目标句子的长度。
        """
        # 对目标句子，要掩盖住embedding矩阵的上半部分（代表从过去向未来询问的部分）
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size()[-1]).to(device)
        # 但除此之外，我们还需要对所有填充的部分进行掩码，减少对模型的噪音干扰
        # 掩盖住原句子中<pad>的部分，例如[[False,False,False,..., True,True,...], ...]
        src_key_padding_mask = TranslationModel.get_key_padding_mask(src)
        # 掩盖住目标句子中<pad>的部分
        tgt_key_padding_mask = TranslationModel.get_key_padding_mask(tgt)

        # 对src和tgt进行编码
        src = self.src_embedding(src)
        tgt = self.tgt_embedding(tgt)
        # 给src和tgt的token增加位置信息
        src = self.positional_encoding(src)
        tgt = self.positional_encoding(tgt)
        # 经过这一步之后，数据的结构变成了三维
        #(batch_size, sentence_len, embedding_dimension)

        # 将准备好的数据送给nn.transformer
        # 在这里我们是一并将源数据与目标数据给到了transformer模型
        # 但transformer模型实际上是先试用src原始数据
        # 再试用tgt目标数据的
        # 同时两个数据的掩码也各不相同
        out = self.transformer(src, tgt,
                               tgt_mask=tgt_mask,
                               src_key_padding_mask=src_key_padding_mask,
                               tgt_key_padding_mask=tgt_key_padding_mask)

        """
        这里直接返回transformer的结果。因为训练和推理时的行为不一样，
        所以在该模型外再进行线性层的预测。
        """
        return out

    @staticmethod
    def get_key_padding_mask(tokens):
        """
        用于key_padding_mask
        """
        return tokens == 2

规定训练次数到达设定的次数时，就保存一次模型：

In [62]:
if model_checkpoint:
    model = torch.load(model_dir / model_checkpoint)
else:
    model = TranslationModel(256, en_vocab, zh_vocab)
model = model.to(device)

检验一下模型能够运行：

In [63]:
model(src, tgt).size()

torch.Size([64, 71, 256])

In [64]:
torch.Size([64, 71, 256])

torch.Size([64, 71, 256])

In [65]:
model(src, tgt)

tensor([[[-0.1086, -0.9489,  0.0671,  ...,  0.2900, -1.0769, -0.9015],
         [ 0.7002,  0.2061, -0.3974,  ...,  0.7878, -0.9466, -0.8742],
         [ 0.4416,  0.5567,  0.8869,  ...,  0.0506, -1.1705, -0.9455],
         ...,
         [ 0.5505,  0.4742,  0.7696,  ...,  0.2593, -0.0192, -0.9401],
         [ 0.5870,  0.0738,  0.4658,  ...,  1.6159, -0.9348, -0.8466],
         [ 0.5404,  0.3085,  0.8862,  ...,  1.2423, -0.3007, -0.4248]],

        [[ 0.2161, -0.4012,  0.1673,  ...,  0.9661,  0.0261, -0.4047],
         [-1.0799,  0.6539,  0.4570,  ...,  0.7826, -0.3653, -1.0629],
         [ 0.6776, -1.7098,  0.1928,  ...,  1.0279, -0.2207, -0.3342],
         ...,
         [ 0.6773,  0.4844, -0.0996,  ...,  0.7487, -0.1460, -1.2397],
         [ 1.0414, -1.3704, -0.5414,  ...,  0.5545, -0.5459,  0.0381],
         [ 1.0267, -0.6749,  0.1335,  ...,  0.1397, -0.0418, -0.6256]],

        [[ 0.8706,  0.0169, -0.6459,  ...,  1.9233,  0.4750, -0.3505],
         [ 1.0289,  0.0775, -1.1608,  ...,  1

## 实际训练过程

### 优化器与损失函数

In [66]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [67]:
class TranslationLoss(nn.Module):

    def __init__(self):
        super(TranslationLoss, self).__init__()
        # 使用KLDivLoss，不需要知道里面的具体细节。
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = 2

    def forward(self, x, target):
        """
        损失函数的前向传递
        :param x: 将Decoder的输出再经过predictor线性层之后的输出。
                  也就是Linear后、Softmax前的状态
        :param target: tgt_y。也就是label，例如[[1, 34, 15, ...], ...]
        :return: loss
        """

        """
        由于KLDivLoss的input需要对softmax做log，所以使用log_softmax。
        等价于：log(softmax(x))
        """
        x = log_softmax(x, dim=-1)

        """
        构造Label的分布，也就是将[[1, 34, 15, ...]] 转化为:
        [[[0, 1, 0, ..., 0],
          [0, ..., 1, ..,0],
          ...]],
        ...]
        """
        # 首先按照x的Shape构造出一个全是0的Tensor
        true_dist = torch.zeros(x.size()).to(device)
        # 将对应index的部分填充为1
        true_dist.scatter_(1, target.data.unsqueeze(1), 1)
        # 找出<pad>部分，对于<pad>标签，全部填充为0，没有1，避免其参与损失计算。
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)

        # 计算损失
        return self.criterion(x, true_dist.clone().detach())

In [68]:
criteria = TranslationLoss()

In [69]:
writer = SummaryWriter(log_dir='runs/transformer_loss')

In [29]:
torch.cuda.empty_cache()

### 实际训练流程与代码

In [ ]:
step = 0

if model_checkpoint:
    step = int('model_10000.pt'.replace("model_", "").replace(".pt", ""))

model.train()
for epoch in range(epochs):
    loop = tqdm(enumerate(train_loader), total=len(train_loader))
    for index, data in enumerate(train_loader):
        # 生成数据
        src, tgt, tgt_y, n_tokens = data
        src, tgt, tgt_y = src.to(device), tgt.to(device), tgt_y.to(device)

        # 清空梯度
        optimizer.zero_grad()
        # 进行transformer的计算
        out = model(src, tgt)
        # 将结果送给最后的线性层进行预测
        out = model.predictor(out)

        """
        计算损失。由于训练时我们的是对所有的输出都进行预测，所以需要对out进行reshape一下。
                我们的out的Shape为(batch_size, 词数, 词典大小)，view之后变为：
                (batch_size*词数, 词典大小)。
                而在这些预测结果中，我们只需要对非<pad>部分进行，所以需要进行正则化。也就是
                除以n_tokens。
        """
        loss = criteria(out.contiguous().view(-1, out.size(-1)), tgt_y.contiguous().view(-1)) / n_tokens
        # 计算梯度
        loss.backward()
        # 更新参数
        optimizer.step()

        loop.set_description("Epoch {}/{}".format(epoch, epochs))
        loop.set_postfix(loss=loss.item())
        loop.update(1)

        step += 1

        del src
        del tgt
        del tgt_y

        if step != 0 and step % save_after_step == 0:
            torch.save(model, model_dir / f"model_{step}.pt")

Epoch 0/1:   1%|▏         | 2313/156250 [3:34:59<243:49:26,  5.70s/it, loss=4.24]

还记得吗？我们设置了epoch为1。运行一个epoch就需要3个半小时的时间，你可以尝试着在GPU上运行大约3~5个epoch看看能否有更好的效果↑ 

## 推理与测试

我们直接使用从外部保存的模型，有我们训练好的5500模型，你也可以使用你自己训练好的模型。

In [70]:
model = torch.load('model_5000.pt', map_location=torch.device('cpu'))

In [71]:
model = model.eval()

In [72]:
def translate(src: str):
    """
    :param src: 英文句子，例如 "I like machine learning."
    :return: 翻译后的句子，例如：”我喜欢机器学习“
    """

    # 将与原句子分词后，通过词典转为index，然后增加<bos>和<eos>
    src = torch.tensor([0] + en_vocab(en_tokenizer(src)) + [1]).unsqueeze(0).to(device)
    # 首次tgt为<bos>
    tgt = torch.tensor([[0]]).to(device)
    # 一个一个词预测，直到预测为<eos>，或者达到句子最大长度
    for i in range(max_length):
        # 进行transformer计算
        out = model(src, tgt)
        # 预测结果，因为只需要看最后一个词，所以取`out[:, -1]`
        predict = model.predictor(out[:, -1])
        # 找出最大值的index
        y = torch.argmax(predict, dim=1)
        # 和之前的预测结果拼接到一起
        tgt = torch.concat([tgt, y.unsqueeze(0)], dim=1)
        # 如果为<eos>，说明预测结束，跳出循环
        if y == 1:
            break
    # 将预测tokens拼起来
    tgt = ''.join(zh_vocab.lookup_tokens(tgt.squeeze().tolist())).replace("<s>", "").replace("</s>", "")
    return tgt

In [73]:
translate("Alright, this project is finished. Let's see how good this is.")

'好，这项目结束了。让我们看看看这个是多好的。'

In [76]:
translate("How are you?")

'你好吗？'

In [75]:
translate("Today is a nice day!")

'今天是一个美好的日子！'

In [78]:
translate("I have many friends.")

'我有很多朋友。'

In [81]:
translate("Do you have any problems?")

'你有什么问题吗？'

In [89]:
translate("What can I say? Wish you a nice day!")

'我能说什么？我可以说什么？祝你今天愉你好一天！'

你也可以从数据集中分割出训练和测试集（例如分割出100w测试集），将测试集用于单独的测试来验证模型的效果。在机器翻译的过程中，数据准备是比模型训练更为关键的流程，当你在你的数据集上进行了完备的准备，你的训练流程也会更加流畅。你可以尝试更换数据集、在你自己的数据集上进行测试。

# Transformer案例实战之 股价预测

股价预测是利用计算机程序通过分析历史和实时的市场数据来预测未来股票价格的技术，它是时序领域中最常见应用之一，也是因其巨大潜力而备受关注的领域。这个任务涉及到从历史股价数据中提取趋势，并将这些趋势准确地映射到未来的预测中，同时尽可能地捕捉市场动态和投资者情绪的变化。在使用深度学习技术进行股价预测的各种实践当中，Transformer被认为是最为强大的实践模型，Transofrmer通过训练大量的股市交易数据来学习价格波动的复杂模式、ta能够理解和预测股票价格的复杂波动，适应不同的市场条件，并处理非线性的市场因素。股价预测是金融技术领域中的一个重要而活跃的研究方向，广泛应用于投资决策、风险管理和财务规划等领域。

对于Transformer模型来说，股价预测是一个典型的时间序列分析任务。这意味着模型需要接收一系列的输入（如历史股价和交易量的数据），并输出另一系列的数据（未来的股价预测）。Transformer中的自注意力机制允许模型在预测时能够关注到输入序列中的所有数据点，帮助模型理解更复杂的市场趋势和周期性波动，使得预测更加准确和具有前瞻性；同时，在股价预测任务中，Transformer中的编码器用于分析输入的市场数据，而解码器则用于生成未来价格的预测。每一层都进一步提炼和传递信息，增强了模型在处理复杂市场数据时的能力。

在之前的课程中，我们使用LSTM完成过高度完整、考虑到方方面面的股价预测流程。**与其他Transformer的任务一样，对股价预测来说，最重要的流程是处理数据**。在高质量的数据上、架构则会决定当前预测的上限，因此我们将复用在LSTM流程中所使用的数据处理流程、除了架构之外的细节也都可以参考LSTM的股价预测案例本身。当然，当Transformer在执行股价预测时，会存在许多与LSTM不同的地方，包括对未来信息进行掩码、需要同时使用过去的信息和未来的信息等等。在本次案例中，我们将具体使用代码来实现这些不同之处。

今天的项目中，我们将完成一个基于历史数据的股价预测任务。通常来说，一个股价预测任务需要覆盖至少如下的流程：

- 一、数据处理

1. **加载数据**：股票数据的加载可以使用一个很复杂的工作，对于单量数据集来说、只要从CSV文件中读取股票的收盘价数据、并要求该数据按照时间顺序进行排列即可；但对于多变量数据集、甚至面板数据来说，在导入数据后则要确定数据是如何排列的、按照时间顺序排列、与按照股票顺序排列后再按时间顺序排列，其实大有不同。

2. **股票数据的预处理**：在任意股票数据上，我们都可以使用下面的这些预处理手段——
> `MinMaxScaler归一化`：使用`MinMaxScaler`归一化，使数据值在-1到1之间，以便于神经网络处理。在之前的课程中我们就提到过，归一化对股票数据而言是一个对模型结果提升巨大的预处理流程。即便不进行任何特征工程，只要能够对数据进行归一化，对数据预测也有很大的帮助。当然，归一化具体怎么做也会很大程度影响模型的表现，我们可以针对每个股票进行精确的、属于单支股票自己的归一化，也可以就对全部数据进行笼统的简单归一化，有时候简单归一化反而能够达到抗过拟合的效果，需要根据实际的需求进行判断。另外，是否需要对标签本身（也就是预测的股价本身）进行归一化也是需要讨论的命题。通常来说，为了保证预测损失的可解释性，我们不会对标签本身也进行归一化，但从神经网络的预测角度来看，我们可以对股票数据的标签进行归一化，如果你在预测的不是股价本身、而是某种趋势、某种比例，那你完全可以对股价本身也进行归一化来加速Transformer的运行。<br><br>
> `调价因子`：在许多股票数据集中，都会有`Dividend`、市场监管、`AdjustmentFactor`等特征，这些特征对于股票价格有一定的影像，因此我们需要根据金融业务、依据这些特征对股票价格进行调整，等到调整后的价格。<br><br>
> `数据分割`：数据被分为训练集和测试集。通常，训练集用于模型学习，而测试集用于评估模型的泛化能力。注意，对于股票数据来说，我们分割的是完全的时间序列数据，因此只能够按照“过去是训练集、未来是测试集”的方式进行分割。当然，如果我们使用的是多变量的数据、甚至是截面的数据，那我们就需要先将每支股票进行分割后、再在每支股票内部进行“过去是训练集、未来是测试集”的分割了。这种方式可以帮助我们更好地预测每一支股票。当然，在股票数量特别多的情况下，对每一支股票进行精准预测可能会造成模型过拟合、或者训练混乱，因此我们也可能将数据更粗糙地分割（例如，按照股票所在的sector进行分割、按照股票所在的行业进行分割等等）。<br><br>
> 在进行预处理的时候，预处理的顺序有很大的学问。我们应该先对数据进行处理再进行数据集分割，还是先分割后再进行数据预处理？最标准的方式是后者，这可以避免测试集的信息泄露到训练集中。如果先做数据处理再进行分割，可能导致模型的表现虚高。
 
3. **创建序列/滑窗**：时间序列数据往往是按照二维表方式进行排列的，但Transformer等系列NLP算法要求的数据输入格式为三维数据。对于文字数据，通常我们使用embedding的手段来将文字二维表转化为三维的embedding数据集，但对于时间序列数据，我们需要进行滑窗、手动创建三维数据集。在之前的LSTM课程中我们详细讲解了对单变量数据集、多变量数据集以及面板数据集进行滑窗的具体手段，同时我们提到了基于特征进行滑窗、包括标签进行滑窗等滑窗方法。按照预测的步数，还可以将预测分为多步预测、递归预测等实现手段。对Transformer而言，我们在进行时间序列预测时，可以采用和LSTM相类似的滑窗方式，但具体的来说——
> 如果我们使用的是完整的**带有编码器和解码器双重结构的Transformer**，那我们可能不需要选择带标签的滑窗，因为解码器本身就要求真实标签作为输入的一部分，我们没有必要再特地将真实标签包括在编码器的输入数据中。如果我们选择了带标签的滑窗，我们可能需要对编码器的数据进行掩码。<br><br>
> 如果我们使用的是**Encoder-only（只有编码器中有多头注意力机制）的Transformer**，那我们可以选择和LSTM一样的、带标签的滑窗方式来提升模型的效果。

由于在LSTM的案例中我们已经非常详细地解读过各类时间序列数据、以及各类滑窗方式的详细讲解，因此在这里不再赘述。如果你对滑窗方式没有足够的了解，请回看LSTM课程的内容。

- 二、模型定义

在传统的Transformer架构中，编码器和解码器的概念是从机器翻译的角度来定义的，其中编码器处理输入序列，而解码器生成输出序列。对于股价预测这样的时间序列任务，应用Transformer可能需要一些调整，因为股价预测不完全符合典型的序列到序列模型（Seq2Seq）的框架。在这里我们可能需要讨论，**时间序列预测任务是更像分类/回归任务，还是更像生成式任务**？这个答案可能因任务而异，但对于股票场景来说，**股票预测不是一次性对未来进行超长时间段的预测，相反，股票预测是通过滑窗的方式，不断利用更靠近未来的历史信息去预测固定时间段外的未来的任务**，这是说，我们有两种预测类型：

> 1. 利用过去120天的数据预测未来300天的任务

> 2. 将过去120天数据划分为30天一个窗用1-30天的窗去预测31-37天，用2-31天的窗去预测32-38天，这样的任务

很显然，文本生成任务时我们使用的是第一种预测类型，我们需要根据过去少量的信息预测大量的未来，通常预测序列会非常长。但在进行股票预测时，大多数时候我们需要预测短时间内变幻莫测的股票市场，因此股票预测场景下的多步预测往往时间区间不会很长，一般是5-7天，最长差不多是30天。相比起一次性要生成几百字的生成式算法、我们更倾向于认为股票预测是回归任务。

当我们把股票预测当做时回归任务来看待时，我们可以使用“encoder-only”的结构，即只使用encoder来进行预测，在decoder的部分我们使用线性层来帮助我们进行输出。如此，decoder部分复杂的掩码工作、teacher forcing工作等等都无需出现在股票预测的场景中。当然，如果你非常想要使用完整的encoder-decoder结构的transformer，你也可以参考机器翻译案例中的代码来进行修改。

1. **位置编码（`PositionalEncoding`）**：虽然Transformer对数据是一个、一个进行预测，但并不代表训练的时候这些数据是一个个进入网络的；和LSTM、RNN等线性的架构不同，Transformer的会将时间序列上所有的点混合在一起打包成QKV矩阵，并同时计算每一个时间串所对应的未来，因此进入Transformer的时间序列数据也需要被添加位置信息，来帮助Transformer认知到原始数据的顺序。幸运的是，这里进行位置编码的方式和文字数据是一模一样的，我们就不再进行赘述。

2. **Transformer模型（`TransAm`）**：在这里我们使用了encoder-only的Transformer，包括位置编码层、多层Transformer编码器和一个使用线性层填充的decoder作为输出层。这里的decoder因为没有包含多头注意力机制，因此不能够算是使用了经典的encoder架构。本质我们使用的还是encoder-only。

需要注意的是，在本次的例子中我们进行的是单步预测（也就是每次预测只输出未来的1个样本的预测），这是为了让Transformer代码本身看起来不要变得更加复杂。如果你希望进行多步预测，则需要修改参数pred_len，同时别忘记在训练结束后、要对训练出的结果进行去重、才能获得最终的预测结果。当然，如果你使用的是encoder-decoder双重架构，那你可能滑窗方式与现在课程中所设置的有所区别（例如，你没有采用逐步输出的方式，而是直接把超长序列放入encoder，然后输出未来的超长序列），这种情况下你可能无需设置pred_len，而是采用与机器翻译更相似的方法来进行预测。如果你使用encoder-decoder结构，别忘记对序列中未来的信息进行掩码哦。

5. **权重初始化**：选择合适的初始化方法对于模型的学习效率和最终性能非常关键。通过初始化偏置为 0 和权重为一个较小的均匀分布，可以帮助避免训练初期的梯度消失或爆炸问题，使得模型训练过程更加稳定。这种初始化策略尤其在处理复杂的模型如深层神经网络时非常重要。

在这里，我们只对decoder模型使用了均匀分布初始化，因为编码器部分我们使用了PyTorch中的TransformerEncoderLayer测过，这个层默认已经包括了权重初始化，通常是Glorot（也称为Xavier）初始化或其他类型，因此无需再进行初始化了。同样的，如果你使用的是完整的encoder-decoder结构，那你可能也无需再进行初始化了。

- 三、训练过程

股票数据的训练过程也相对单纯，这一次我们使用了batch循环和epoch循环分割的写法，我们将batch循环打包在了一个函数当中，将epoch循环暴露在外，这样分开的目的是为了让训练代码更加简洁、也是为大家提供了一套全新的神经网络训练代码。

1. **定义损失函数和优化器**：使用均方误差损失函数（MSE）和AdamW优化器。
2. **学习率调整**：使用学习率调度器来调整学习率。
3. **训练循环**：在每个epoch中，模型在训练数据上进行迭代，计算损失，进行反向传播和参数更新。

- 四、评估和可视化
1. **评估函数**：定义了一个评估函数来计算模型在测试集上的总损失。
2. **可视化结果**：如果你是对单支股票进行预测、那在特定的epoch后，可以使用`plot_and_loss`函数绘制模型预测的结果和实际值，以可视化模型的性能。在我们的例子中，我们是对抽选的多支股票进行预测，因此就没有进行绘图。

接下来就让我们一步步来完成这个案例。

## 数据准备

In [123]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os
from decimal import ROUND_HALF_UP, Decimal
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import torch
import torch.nn as nn
import time
import math
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.functional import pad, log_softmax

# typing 模块提供了一些类型，用于类型提示
from typing import Union,List,Tuple,Iterable

### 环境设置与数据导入

In [124]:
# 工作目录、为Transformer放数据、缓存文件的目录
work_dir = Path(r"D:\pythonwork\2024DL")
# 训练好的模型会放在该目录下，注意隔一段时间就要对模型进行保存，这是深度学习训练的基本
model_dir = Path(r"D:\pythonwork\2024DL\model")
# 上次运行到的地方，如果是第一次运行，为None，如果中途暂停了，下次运行时，指定目前最新的模型即可。
model_checkpoint = None # 'model_10000.pt'

# 如果工作目录不存在，则创建一个
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

# 如果模型目录不存在，则创建一个
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# 股价数据本身的导入路径
stock_price_filepath = './stock_prices.csv'
# 股价列表的导入路径
stock_list_filepath = './stock_list.csv'

In [240]:
#数据导入
stock= pd.read_csv(stock_price_filepath)
stock_list = pd.read_csv(stock_list_filepath)

In [241]:
#查看数据的具体情况

In [242]:
stock.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [243]:
stock_list.head()

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,1.092828e+07,3.365911e+10,True
1,1305,20211230,Daiwa ETF-TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2097.0,3.634636e+09,7.621831e+12,False
2,1306,20211230,NEXT FUNDS TOPIX Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2073.5,7.917718e+09,1.641739e+13,False
3,1308,20211230,Nikko Exchange Traded Index Fund TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2053.0,3.736943e+09,7.671945e+12,False
4,1309,20211230,NEXT FUNDS ChinaAMC SSE50 Index Exchange Trade...,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,44280.0,7.263200e+04,3.216145e+09,False


In [244]:
#为了效率我们抽取其中的10支股票
#你可以更换随机数种子，当你更换后，你选出的5支股票可能与我不一样

# 从 SecuritiesCode 中随机选择5个不同的股票代码
selected_codes = stock['SecuritiesCode'].drop_duplicates().sample(n=5,random_state=1412)

# 根据选中的股票代码筛选出所有对应的数据行
stock = stock[stock['SecuritiesCode'].isin(selected_codes)]

In [245]:
stock["SecuritiesCode"].unique() #1429, 6902, 1884, 8360, 7282 确保和之前取出的一样

array([1429, 1884, 6902, 7282, 8360], dtype=int64)

In [246]:
stock = stock.sort_values("SecuritiesCode")

In [247]:
#恢复索引
stock.index = range(stock.shape[0])

In [248]:
stock.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1429,2017-01-04,1429,418.0,420.0,418.0,418.0,24900,1.0,NaN,False,0.007160
1,20210729_1429,2021-07-29,1429,534.0,538.0,533.0,538.0,22600,1.0,NaN,False,0.016729
2,20181004_1429,2018-10-04,1429,402.0,402.0,391.0,398.0,53900,1.0,NaN,False,-0.012853
3,20200630_1429,2020-06-30,1429,600.0,603.0,587.0,591.0,60000,1.0,NaN,False,0.013514
4,20171106_1429,2017-11-06,1429,509.0,512.0,503.0,504.0,31100,1.0,NaN,False,0.013972


### 股票数据的数据预处理

In [249]:
#预处理函数
def preprocess_stock_data(stock_data):
    """
    对股票数据进行预处理，包括重命名列、重新排序列、填补缺失值、删除具有大量缺失值的行以及重置索引。
    
    :param stock_data: DataFrame类型，需要预处理的股票数据。
    :return: 预处理后的股票数据。
    """
    # 创建数据的副本以避免修改原始数据
    processed_data = stock_data.copy()
    
    # 将Target列重命名为Sharpe Ratio
    processed_data.rename(columns={'Target': 'Sharpe Ratio'}, inplace=True)
    
    # 填补ExpectedDividend列的缺失值
    processed_data["ExpectedDividend"] = processed_data["ExpectedDividend"].fillna(0)
    
    # 删除具有大量缺失值的行
    processed_data.dropna(inplace=True)
    
    # 重置索引
    processed_data.index = range(processed_data.shape[0])
    
    return processed_data

In [250]:
#分割函数
def split_data_by_column(data, column_name, train_ratio=0.67):
    """
    按照指定列的值分割数据集为训练集和测试集。
    
    :param data: DataFrame类型，需要分割的数据集。
    :param column_name: str类型，用作分割依据的列名。
    :param train_ratio: float类型，训练集所占比例，默认值为0.67。
    :return: 分割后的训练集和测试集。
    """
    train = pd.DataFrame()
    test = pd.DataFrame()

    for value in data[column_name].unique():
        subset = data[data[column_name] == value]
        train_size = int(len(subset) * train_ratio)
        train = pd.concat([train, subset[:train_size]])
        test = pd.concat([test, subset[train_size:]])
    
    #由于我们是按照指定列将train和test分开
    #因此现在train和test的索引被切断了
    #需要为train、test恢复索引
    
    train.index = range(train.shape[0])
    test.index = range(test.shape[0])
    
    return train, test

In [251]:
#精密的归一化处理
def normalize_data_bycode(train, test, columns_to_normalize):
    """
    对指定列的数据进行归一化处理。
    
    :param train_data: DataFrame类型，训练数据集。
    :param test_data: DataFrame类型，测试数据集。
    :param columns_to_normalize: list类型，需要归一化的列名列表。
    :return: 归一化后的训练数据集和测试数据集。
    """
    
    for sec_code in train['SecuritiesCode'].unique():
        
        train.loc[train['SecuritiesCode'] == sec_code, columns_to_normalize] = train.loc[train['SecuritiesCode'] == sec_code, columns_to_normalize].astype(float)
        test.loc[test['SecuritiesCode'] == sec_code, columns_to_normalize]  = test.loc[test['SecuritiesCode'] == sec_code, columns_to_normalize].astype(float)
        
        scaler = MinMaxScaler()
        
        # 训练集归一化
        scaler.fit(train.loc[train['SecuritiesCode'] == sec_code, columns_to_normalize])
        train.loc[train['SecuritiesCode'] == sec_code, columns_to_normalize] = scaler.transform(train.loc[train['SecuritiesCode'] == sec_code, columns_to_normalize])
        
        # 测试集归一化，注意这里使用与训练集相同的scaler
        if sec_code in test['SecuritiesCode'].values:  # 确保测试集中存在该股票代码
            test.loc[test['SecuritiesCode'] == sec_code, columns_to_normalize] = scaler.transform(test.loc[test['SecuritiesCode'] == sec_code, columns_to_normalize])
    
    return train, test

#粗糙的归一化处理
def normalize_data_byall(train, test, columns_to_normalize):
    scaler = MinMaxScaler()
        
    # 训练集归一化
    scaler.fit(train.loc[:,columns_to_normalize])
    train.loc[:, columns_to_normalize] = scaler.transform(train.loc[:, columns_to_normalize])
        
    # 测试集归一化，注意这里使用与训练集相同的scaler
    test.loc[:, columns_to_normalize] = scaler.transform(test.loc[:, columns_to_normalize])
    
    return train, test

In [252]:
# 定义基于AdjustmentFactor的调价函数
def adjust_price(price):
    """
    参数：
    price (pd.DataFrame) : 包含股票价格信息的 pd.DataFrame。

    返回：
    price DataFrame (pd.DataFrame): 含有新生成的 AdjustedClose 的股票价格DataFrame

    函数功能：
    该函数将输入的原始股价数据进行处理，生成带有调整后收盘价(AdjustedClose)的新DataFrame。
    AdjustedClose的计算方式为原收盘价与调整因子(AdjustmentFactor)的累计乘积，结果保留一位小数。
    若计算后的AdjustedClose为0，则替换为空值(np.nan)，并对此列数据进行向前填充，以保持数据完整性。
    """
    # 将 Date 列转换为 datetime 格式
    # 因为我们在前面查看数据类型的时候发现Date为object类型
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        参数:
            df (pd.DataFrame)  : 单个 SecuritiesCode 的股票价格
        返回:
            df (pd.DataFrame): 单个 SecuritiesCode 的带有 AdjustedClose 的股票价格
        """
        # 排序数据以生成 CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # 生成 CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()#累乘
        # 生成 AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # 反转顺序
        df = df.sort_values("Date")
        # 填充 AdjustedClose，将 0 替换为 np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # 向前填充 AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df
    
    # 生成 AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)
    return price

In [253]:
#完成除了移动标签列和归一化之外的全部预处理

#数据预处理
data_ = preprocess_stock_data(stock)
# 行业可能会对股票有影响，所以将行业也加入特征进行建模
# 将17SectorName进行labelencoder
stock_list['17SectorName'] = LabelEncoder().fit_transform(stock_list['17SectorName'])
# 并入主表
data_ = data_.merge(stock_list[['SecuritiesCode','17SectorName']],on='SecuritiesCode')
# 调整股价
data_ = adjust_price(data_)
#将close列移动到最后一列
close_col = data_.pop('Close')
data_.loc[:,'Close'] = close_col
#将所有除了时间、id之外的数据转换成浮点数
for column in data_.columns[3:]:
    data_[column] = data_[column].astype(float)
#分割数据
train, test = split_data_by_column(data_,"SecuritiesCode")

In [254]:
#粗糙的数据归一化
unnormalized_close_train = train.loc[:,"Close"].copy()
unnormalized_close_test = test.loc[:,"Close"].copy()

train, test = normalize_data_byall(train,test
                            ,columns_to_normalize = train.columns[3:])

In [255]:
train.head() #确认是按股票顺序排列、且每支股票内部是时间顺序，才能开始滑窗

,RowId,Date,SecuritiesCode,Open,High,Low,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Sharpe Ratio,17SectorName,CumulativeAdjustmentFactor,AdjustedClose,Close
0,20170104_1429,2017-01-04 00:00:00,1429,0.012889,0.012586,0.013510,0.003915,0.0,0.0,0.0,0.392311,1.0,0.0,0.012652,0.012652
1,20170105_1429,2017-01-05 00:00:00,1429,0.013149,0.012586,0.013240,0.005083,0.0,0.0,0.0,0.343628,1.0,0.0,0.012783,0.012783
2,20170106_1429,2017-01-06 00:00:00,1429,0.012759,0.012845,0.013375,0.006731,0.0,0.0,0.0,0.376025,1.0,0.0,0.013179,0.013179
3,20170110_1429,2017-01-10 00:00:00,1429,0.013280,0.013105,0.013510,0.011693,0.0,0.0,0.0,0.327266,1.0,0.0,0.012783,0.012783
4,20170111_1429,2017-01-11 00:00:00,1429,0.013019,0.012716,0.013105,0.005804,0.0,0.0,0.0,0.384325,1.0,0.0,0.012915,0.012915


In [259]:
# 注意我们不要归一化后的target
# 将保存的Close列放回DataFrame中

close_col = train.pop("Close")
train.loc[:,"Close"] = unnormalized_close_train

close_col = test.pop("Close")
test.loc[:,"Close"] = unnormalized_close_test

### 关注截面的、基于特征+标签的滑窗

In [260]:
#关注截面的滑窗

def create_multivariate_dataset_4(dataset, window_size, pred_len):
    """
    将多变量时间序列转变为能够用于训练和预测的数据，确保每个窗口内的Securities Code唯一
    
    参数:
        dataset: DataFrame，其中包含特征和标签，特征从索引3开始，最后一列是标签
        window_size: 滑窗的窗口大小
    """
    X, y, y_indices = [], [], []
    for i in range(len(dataset) - window_size - pred_len + 1):
        # 检查窗口内的Securities Code是否唯一
        securities_code = dataset.iloc[i:i + window_size, 2]
        if len(securities_code.unique()) == 1:  # 如果Securities Code在窗口内唯一
            # 选取从第4列到最后一列的特征和标签
            feature_and_label = dataset.iloc[i:i + window_size, 3:].copy().values
            # 长度pred_len的标签作为目标
            target = dataset.iloc[(i + window_size):(i + window_size + pred_len), -1]
            
            # 记录本窗口中要预测的标签的时间点
            target_indices = list(range(i + window_size, i + window_size + pred_len))

            X.append(feature_and_label)
            y.append(target)
            #将每个标签的索引添加到y_indices列表中
            y_indices.extend(target_indices)
            
    X = torch.FloatTensor(np.array(X, dtype=np.float32))
    y = torch.FloatTensor(np.array(y, dtype=np.float32))            

    return X, y, y_indices

In [261]:
#设置参数
input_size = train.shape[1]-3  #输入特征的维度
n_head = 6 #多头注意力机制的头数
epochs = 1000 #迭代epoch
learning_rate = 0.01 #学习率
window_size = 30 #窗口大小
pred_len = 1 #多步预测的步数，对于Transformer而言直接执行单步预测也没有问题
num_layers = 1 #编码器/解码器的层数

In [262]:
#对数据进行滑窗
X_train_final, y_train_final, y_train_indices = create_multivariate_dataset_4(train, window_size, pred_len)
X_test_final, y_test_final, y_test_indices = create_multivariate_dataset_4(test, window_size, pred_len)

In [263]:
X_train_final.shape

torch.Size([3874, 30, 12])

In [264]:
X_train_final

tensor([[[1.2889e-02, 1.2586e-02, 1.3510e-02,  ..., 0.0000e+00,
          1.2652e-02, 4.1800e+02],
         [1.3149e-02, 1.2586e-02, 1.3240e-02,  ..., 0.0000e+00,
          1.2783e-02, 4.1900e+02],
         [1.2759e-02, 1.2845e-02, 1.3375e-02,  ..., 0.0000e+00,
          1.3179e-02, 4.2200e+02],
         ...,
         [1.3019e-02, 1.2586e-02, 1.1889e-02,  ..., 0.0000e+00,
          1.1597e-02, 4.1000e+02],
         [1.2238e-02, 1.2326e-02, 1.2834e-02,  ..., 0.0000e+00,
          1.2520e-02, 4.1700e+02],
         [1.2368e-02, 1.2197e-02, 1.2294e-02,  ..., 0.0000e+00,
          1.1729e-02, 4.1100e+02]],

        [[1.3149e-02, 1.2586e-02, 1.3240e-02,  ..., 0.0000e+00,
          1.2783e-02, 4.1900e+02],
         [1.2759e-02, 1.2845e-02, 1.3375e-02,  ..., 0.0000e+00,
          1.3179e-02, 4.2200e+02],
         [1.3280e-02, 1.3105e-02, 1.3510e-02,  ..., 0.0000e+00,
          1.2783e-02, 4.1900e+02],
         ...,
         [1.2238e-02, 1.2326e-02, 1.2834e-02,  ..., 0.0000e+00,
          1.252

In [265]:
y_train_final

tensor([[406.],
        [412.],
        [414.],
        ...,
        [699.],
        [735.],
        [733.]])

## 模型构建与模型训练

### 位置编码函数

In [ ]:
import torch
import numpy as np
import random

def set_seed(seed_value=1):
    random.seed(seed_value)  # Python内置的随机库
    np.random.seed(seed_value)  # Numpy库
    torch.manual_seed(seed_value)  # 为CPU设置种子
    torch.cuda.manual_seed(seed_value)  # 为当前GPU设置种子
    torch.cuda.manual_seed_all(seed_value)  # 为所有GPU设置种子
    torch.backends.cudnn.deterministic = True  # 确保每次返回的卷积算法将是确定的
    torch.backends.cudnn.benchmark = False

set_seed(2)

# 定义了学习率调度
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

# 定义位置编码模块
class PositionalEncoding(nn.Module):
    # 初始化函数，设置模型参数
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        # 创建位置编码张量，全零初始化
        pe = torch.zeros(max_len, d_model)
        # 生成位置序列
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 计算分母项，用于调整正弦和余弦函数的频率
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # 使用正弦函数生成位置编码的偶数部分
        pe[:, 0::2] = torch.sin(position * div_term)
        # 使用余弦函数生成位置编码的奇数部分
        pe[:, 1::2] = torch.cos(position * div_term)
        # 修改pe形状并固定维度顺序
        pe = pe.unsqueeze(0).transpose(0, 1)
        # 注册位置编码为模型的缓冲区
        self.register_buffer('pe', pe)

    # 前向传播函数，将位置编码添加到输入张量上
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

### Encoder-Only的Transformer结构

In [ ]:
# 定义Transformer模型
class TransAm(nn.Module):
    # 初始化函数，设置模型参数
    def __init__(self, feature_size=250, pred_len=1, n_head = 1, num_layers=1, dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None  # 源数据掩码，用于遮蔽未来信息
        # 实例化位置编码模块，注意此时因为数据本来就是浮点数，所以无需再次embedding
        self.pos_encoder = PositionalEncoding(feature_size)
        # 创建Transformer编码层
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=n_head, dropout=dropout)
        # 根据编码层构建Transformer编码器
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        # 初始化解码器，将Transformer输出转换为预测长度
        self.decoder = nn.Linear(feature_size, pred_len)
        # 初始化模型权重
        self.init_weights()

    # 初始化权重函数，用于设置解码器的初始权重
    def init_weights(self):
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    # 前向传播函数
    def forward(self, src):
        # 检查并更新掩码
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask
        # 应用位置编码
        src = self.pos_encoder(src)
        # 通过编码器处理数据
        output = self.transformer_encoder(src, self.src_mask)
        # 选择最后一个时间步的输出进行解码
        output = output[:, -1, :]
        # 通过解码器生成最终预测
        output = self.decoder(output)
        return output

    # 生成后续掩码，用于遮蔽未来信息
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [274]:
#设置GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#实例化模型、进行训练
model = TransAm(input_size, pred_len, n_head, num_layers).to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate) #定义优化器
criterion = nn.MSELoss() #定义损失函数
train_loader = data.DataLoader(data.TensorDataset(X_train_final, y_train_final)
                         #每个表单内部是保持时间顺序的即可，表单与表单之间可以shuffle
                         , shuffle=True
                         , batch_size=32
                         , drop_last = True) #将数据分批次
test_loader = data.DataLoader(data.TensorDataset(X_test_final, y_test_final)
                         , shuffle=True
                         , batch_size=32
                         , drop_last = True) #将数据分批次

# 初始化早停参数
early_stopping_patience = 3  # 设置容忍的epoch数，即在这么多epoch后如果没有改进就停止
early_stopping_counter = 0  # 用于跟踪没有改进的epoch数
best_train_rmse = float('inf')  # 初始化最佳的训练RMSE

cuda


## 训练流程

### train_batch函数与epoch分割

In [275]:
def train_batch(trainloader):
    model.train()
    total_loss = 0.
    
    for X_batch, y_batch in trainloader:
        optimizer.zero_grad()
        y_pred = model(X_batch.to(device))
        loss = criterion(y_pred, y_batch.to(device))
        loss.backward()
        #梯度裁剪
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.7)
        optimizer.step()

def evaluate(eval_model, testloader):
    eval_model.eval()
    total_loss = 0
    eval_batch_size = 1000
    with torch.no_grad():
        for X_batch, y_batch in testloader:
            y_pred = eval_model(X_batch.to(device))
            total_loss += len(data[0]) * criterion(y_pred, y_batch.to(device)).cpu().item()
    return total_loss / len(data_source)

In [276]:
def plot_and_loss(eval_model, data_source, epoch):
    eval_model.eval()
    total_loss = 0.
    test_result = torch.Tensor(0)
    truth = torch.Tensor(0)
    with torch.no_grad():
        for X_batch, y_batch in testloader:
            y_pred = eval_model(X_batch.to(device))
            total_loss += len(data[0]) * criterion(y_pred, y_batch.to(device)).cpu().item()
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0)
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)

    plt.plot(test_result, color="red")
    plt.plot(truth, color="blue")
    plt.grid(True, which='both')
    plt.axhline(y=0, color='k')
    plt.savefig('transformer-epoch%d.png' % epoch)
    plt.close()

    return total_loss / i

In [277]:
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train_batch(train_loader)
    #验证与打印
    if epoch % 50 == 0:
        model.eval()
        with torch.no_grad():
            y_pred = model(X_train_final.to(device)).cpu()
            train_rmse = np.sqrt(loss_fn(y_pred, y_train_final))
            y_pred = model(X_test_final.to(device)).cpu()
            test_rmse = np.sqrt(loss_fn(y_pred, y_test_final))
        print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f}'.format(epoch, (
                time.time() - epoch_start_time), test_rmse))
        print('-' * 89)
        scheduler.step()

Epoch 50: train RMSE 463.2390, test RMSE 1203.4922
-----------------------------------------------------------------------------------------
| end of epoch  50 | time:  0.84s | valid loss 1203.49219
-----------------------------------------------------------------------------------------


D:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 100: train RMSE 642.2194, test RMSE 1043.2932
-----------------------------------------------------------------------------------------
| end of epoch 100 | time:  0.90s | valid loss 1043.29321
-----------------------------------------------------------------------------------------
Epoch 150: train RMSE 1590.6501, test RMSE 1939.3271
-----------------------------------------------------------------------------------------
| end of epoch 150 | time:  0.85s | valid loss 1939.32715
-----------------------------------------------------------------------------------------
Epoch 200: train RMSE 1403.1254, test RMSE 1548.5898
-----------------------------------------------------------------------------------------
| end of epoch 200 | time:  0.84s | valid loss 1548.58984
-----------------------------------------------------------------------------------------
Epoch 250: train RMSE 1140.5841, test RMSE 1369.9457
---------------------------------------------------------------------------

在进行大约550个迭代后，我们达到了1178.87537这个损失水准，在同样的数据集下，这是LSTM需要经过特征工程、以及迭代至少1400个epochs后才能达到的水平。同时，Transformer的运行效率也相当不错，进行50个epochs往往值需要1秒不到的时间。显然，Transformer的学习能力十分强悍。不过，在550个epochs后，出现了梯度消失的现象，导致损失变为了nan。在这次的代码中，我们没有设置提前停止，因此迭代一直进行了下去。限于有限的时间，我们无法在代码结果上为你穷尽最好的可能性，但你可以尝试以下手段来对transformer继续进行改善：

- **继续调整学习率调度**
我们的案例中使用了学习率调度schechuler，但效果并不是非常显著，你可以尝试继续调整参数来改变当前的状况。在使用学习率调度时，你需要注意——
> 温和启动：初期使用较小的学习率，逐渐增大，可以帮助模型在训练初期稳定下来，防止梯度爆炸。<br><br>
> 动态调整：训练过程中根据验证集的表现调整学习率，比如在损失停止下降时降低学习率。

- **增加模型的正则化**
> 权重衰减：通过对模型的参数使用L2正则化来防止过拟合，有时也能帮助控制梯度爆炸。<br><br>
> Dropout：增加或调整Transformer中的dropout比率，可以在训练过程中随机“丢弃”部分的输出，增强模型的泛化能力。

- **修改模型架构或参数**
调整Transformer的层数、头的数量或者模型维度，找到最适合你数据的配置。

- **加强梯度裁剪**
我们现在的梯度裁剪策略比较温和，你可以使用更小的阈值来加强梯度裁剪的强度。

- **继续优化数据、例如尝试对数据完成特征工程**
  
在股价预测等金融时间序列分析任务中，原始数据往往含有丰富但未充分利用的信息。进行有效的特征工程可以帮助模型更好地理解和利用这些信息，从而提高预测的准确性和鲁棒性。你可以利用LSTM课程中所构建的一系列特征工程方法来尝试对数据进行有效的特征工程构建，获悉可以进一步降低Transformer预测的初始损失，并将损失降低到LSTM等算法无法达到的程度。

- **使用完整的、带有decoder结构的Transformer**
  
使用编码器来处理历史输入数据，然后通过解码器逐步生成未来每个时间点的预测，将多步预测的步数扩大、或者直接这种结构特别适用于需要预测未来一系列值的情况。

如果使用完整的encoder-decoder结构，就可以在解码器中实现自回归机制，即每一步生成的输出可以作为下一步的输入，这是目前为止最简单的、实现递归预测的方法，这有助于模型在生成每个后续预测时利用所有之前的预测。如果使用techer forceing机制，让模型在每个后续预测时都利用真实的标签，那Transformer结构的预测准确程度还会大幅提升。只要你认真学习了之前的Transformer原理以及这堂案例课，相信将当前的结构修改为encoder-decoder结构对你来说会比较容易。

到这里，我们对股票案例的讲解就结束了，虽然还留有许多可以去尝试的空间，但当前的案例已经比较完整。你可以继续在此基础上进行尝试和探索，如果你获得了更好的结果，也欢迎在课程中与我进行分享。

**Transformer实战（下）**